# Código 14: Camera

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [2]:
#Bibliotecas
try:
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image
except ImportError:
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    !pip install pyglm
    !pip install pillow
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Câmera"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

Para tratar de câmera, introduz-se conceito de três matrizes.

* Matriz "model": responsável por definir a posição, orientação e escala de um objeto 3D em relação ao seu próprio espaço local (espaço do objeto). Trata-se do produto das matrizes que até então foram tratadas. Não há nada de novo.
* Matriz "view": define a posição e orientação da câmera no espaço 3D. Ela representa a transformação que posiciona a cena em relação à câmera.
* Matriz "projection": responsável por mapear coordenadas 3D para coordenadas 2D na tela. Ela define a perspectiva da cena. Transforma o volume de visualização 3D em um espaço 2D, levando em consideração fatores como a distância dos objetos à câmera.

Como essas matrizes alteraram os vértices, elas são utilizadas no shader de vértices. Vamos utilizar apenas cor no shader de fragmento. Deixaremos textura de lado por ora para que o foco seja nas câmeras

In [42]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec3 position;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Solicitando espaço, compilando e linkando

In [43]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Preparando os dados

Aqui, modelaremos um cubo conforme o código `09_Cubo.ipynb`.

In [44]:
#Criando espaço
vertices = np.zeros(48, [("position", np.float32, 3)])

#Preenchendo as coordenadas
vertices['position'] = [
    
    ### CUBO 1
    # Face 1 do Cubo 1 (vértices do quadrado)
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),

    # Face 2 do Cubo 1
    (+0.2, -0.2, +0.2),
    (+0.2, -0.2, -0.2),         
    (+0.2, +0.2, +0.2),
    (+0.2, +0.2, -0.2),
    
    # Face 3 do Cubo 1
    (+0.2, -0.2, -0.2),
    (-0.2, -0.2, -0.2),            
    (+0.2, +0.2, -0.2),
    (-0.2, +0.2, -0.2),

    # Face 4 do Cubo 1
    (-0.2, -0.2, -0.2),
    (-0.2, -0.2, +0.2),         
    (-0.2, +0.2, -0.2),
    (-0.2, +0.2, +0.2),

    # Face 5 do Cubo 1
    (-0.2, -0.2, -0.2),
    (+0.2, -0.2, -0.2),         
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    
    # Face 6 do Cubo 1
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),           
    (-0.2, +0.2, -0.2),
    (+0.2, +0.2, -0.2),


    #### CUBO 2
    # Face 1 do Cubo 2 (vértices do quadrado)
    (+0.1, +0.1, -0.5),
    (+0.5, +0.1, -0.5),
    (+0.1, +0.5, -0.5),
    (+0.5, +0.5, -0.5),

    # Face 2 do Cubo 2
    (+0.5, +0.1, -0.5),
    (+0.5, +0.1, -0.9),         
    (+0.5, +0.5, -0.5),
    (+0.5, +0.5, -0.9),
    
    # Face 3 do Cubo 2
    (+0.5, +0.1, -0.9),
    (+0.1, +0.1, -0.9),            
    (+0.5, +0.5, -0.9),
    (+0.1, +0.5, -0.9),

    # Face 4 do Cubo 2
    (+0.1, +0.1, -0.9),
    (+0.1, +0.1, -0.5),         
    (+0.1, +0.5, -0.9),
    (+0.1, +0.5, -0.5),

    # Face 5 do Cubo 2
    (+0.1, +0.1, -0.9),
    (+0.5, +0.1, -0.9),         
    (+0.1, +0.1, -0.5),
    (+0.5, +0.1, -0.5),
    
    # Face 6 do Cubo 2
    (+0.1, +0.5, -0.5),
    (+0.5, +0.5, -0.5),           
    (+0.1, +0.5, -0.9),
    (+0.5, +0.5, -0.9)]

### Manipulação dos espaços de dados

In [45]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando. Também captura a posicao de loc_color e loc_mat
loc_position = glGetAttribLocation(program, "position")
loc_color = glGetUniformLocation(program, "color")
loc_mat = glGetUniformLocation(program, "mat_transformation")
glEnableVertexAttribArray(loc_position)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_position, 3, GL_FLOAT, False, stride, offset)

### Eventos para modificar a posição da câmera

Aqui será necessário definir três importantes vetores.
1. Posição da câmera (`cameraPos`): indica a posição da câmera no cenário. Nesse caso, iniciaremos na coordenada `(0.0, 0.0, 1.0)`
2. Frente da câmera (`cameraFront`): direção para a qual a câmera está apontando. Iniciaremos apontando para o centro. Fazemos isso com `(0.0,  0.0, -1.0)`
3. Subida da câmera (`cameraUp`): direção do "para cima" da câmera. Neste caso, naturalmente é `(0.0,  1.0,  0.0)`.

É importante que todos os vetores, inclusive os resultantes das operações entre estes três, tenham módulo unitário. Desta maneira, conseguimos controlar a velocidade da mudança da posição através de uma variável defina apenas para isso: `cameraSpeed`.

**Movimentação para frente ou para trás:**

Para a câmera se mover para frente, basta que a posição da câmera ande na direção da frente de câmera. Podemos controlar a velocidade disso atráves da variável velocidade. Um ponto importante é caso a câmera esteja apontando para um outro lugar (na diagonal, por exemplo). Neste caso, como a direção está sendo controlado pelos eventos de cursor, a própria função cursor se resopnsabiliza de alterar o vetor `cameraFront` para garantir que ele fique sempre na mesma direção para onde a câmera aponta. Para andar para trás, basta fazer a operação de subtração.

**Movimentação para esquerda e direita:**

Para a câmera se mover para direita, basta que ande para a direção do produto vetorial entre a direção da frente e a da subida (que gerará um vetor para direita). O único ponto é que o vetor resultante deve ter módulo unitário. Para andar para esquerda, basta subtrair a posição deste vetor ao invés de somar.

In [46]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0)
cameraFront = glm.vec3(0.0,  0.0, -1.0)
cameraUp    = glm.vec3(0.0,  1.0,  0.0)


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp
    
    cameraSpeed = 0.01
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  WIDTH_WINDOW/2
lastY =  HEIGHT_WINDOW/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)
# Nao entendi a diferença
glfw.set_cursor_pos(window, lastX, lastY)

### Definindo matrizes model, view e projection

In [47]:
def model():
    mat_model = glm.mat4(1.0) # matriz identidade
    mat_model = np.array(mat_model)    
    return mat_model

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), WIDTH_WINDOW/HEIGHT_WINDOW, 0.1, 100.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection


### Criando funções de impressão de objetos

In [48]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

def desenha_cubo1():
    # DESENHANDO O CUBO 1 (vértices de 0 até 23)
    for i in range(0,24,4): # incremento de 4 em 4
        R = (i+1)/24
        G = (i+2)/24
        B = (i+3)/24
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor qualquer com base no i
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4)
    
def desenha_cubo2():
    # DESENHANDO O CUBO 2 (vértices de 24 até 47)
    for i in range(24,48,4): # incremento de 4 em 4
        R = (i+1)/48
        G = (i+2)/48
        B = (i+3)/48
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor qualquer com base no i
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4)

### Exibindo na tela

In [ ]:
glfw.show_window(window)

## Loop principal

In [49]:
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    

    # computando e enviando matrizes Model, View e Projection para a GPU
    mat_model = model()
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    

    # desenhando objetos
   
    desenha_cubo1()
    
    desenha_cubo2()

    
    glfw.swap_buffers(window)

glfw.terminate()